In [1]:
import sys
import numpy as np
import healpy as hp
import matplotlib.pyplot as plt
import pytorch_lightning as pl

sys.path.append("/gpfs02/work/akira.tokiwa/gpgpu/Github/SR-SPHERE")
from scripts.diffusion.models.Unet_base import Unet
from scripts.diffusion.DDPM import DDPM, TimestepSampler
from scripts.maploader.datamodules import DataModule
from scripts.utils.params import set_params
from scripts.utils.run_utils import setup_trainer

/gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
params= set_params(
        n_maps=1,
        nside=512,
        order=8,
        batch_size=4,
        difference=True,
        conditioning="concat",
        norm_type="batch",
        act_type="silu",
        use_attn=False,
        mask=True,
        scheduler="cosine",
        timesteps=2000,
        log_name="test",
        )

In [13]:
pl.seed_everything(1234)
### get training data
dm = DataModule(**params['data'])
dm.setup()

[rank: 0] Global seed set to 1234


loading 1 maps in the directory.
LR data loaded from /gpfs02/work/akira.tokiwa/gpgpu/Github/SR-SPHERE/data/nc128.  Number of maps: 1
HR data loaded from /gpfs02/work/akira.tokiwa/gpgpu/Github/SR-SPHERE/data/nc256.  Number of maps: 1
data divided into 768 patches.
train:validation:test = 614:76:78, batch_size: 4


In [14]:
#get sampler type
sampler = TimestepSampler(timesteps=params['diffusion']['timesteps'])

sampler type: uniform, timesteps: 2000


In [15]:
#get model
unet = Unet(params['data']["nside"], params['data']["order"], **params['architecture'])
model = DDPM(unet, sampler, **params['diffusion'])

We are using Adam with lr = 0.0001, gamma = 0.9999
We are using mask with gaussian noise
The schedule is cosine with s = 0.015


In [16]:
train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()

In [17]:
trainer = setup_trainer(**params['train'])
trainer.fit(model, train_loader, val_loader)

data saved in /gpfs02/work/akira.tokiwa/gpgpu/Github/SR-SPHERE/ckpt_logs
data name: test


/gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:166: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit ...
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_D

Epoch 1:  25%|██▌       | 44/173 [00:33<01:36,  1.33it/s, loss=0.0672, v_num=5] 

/gpfs02/work/akira.tokiwa/gpgpu/anaconda3/envs/pylit/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:48: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
